In [11]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [12]:
# Load the dataset
df= pd.read_csv(r'dataset\Titanic-Dataset.csv')
#Basic preprocessing
df = df[['Survived', 'Pclass','Sex', 'Age', 'SibSp', 'Parch', 'Embarked','Fare']]
df=df.dropna()
# Encode categorical variables
df['Sex']=LabelEncoder().fit_transform(df['Sex'])
df['Embarked']=LabelEncoder().fit_transform(df['Embarked'])
# Split the dataset into features and target variable
X = df.drop('Survived', axis=1).values
y = df['Survived'].values
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  


In [13]:
class TitanicDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(X, dtype=torch.float32)
        self.labels = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
    
train_ds = TitanicDataset(X_train, y_train)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

In [16]:
class TitanicModel(nn.Module):
    def __init__(self):
        super(TitanicModel, self).__init__()
        self.fc1 = nn.Linear(7, 64)  # 7 input features
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)   # Output layer for binary classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Sigmoid activation for binary output
        return x
model= TitanicModel()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
#training loop
num_epochs = 50
for epoch in range(num_epochs):
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features).squeeze()  # Squeeze to match the shape of labels
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/50], Loss: 0.9020
Epoch [2/50], Loss: 0.7869
Epoch [3/50], Loss: 0.3872
Epoch [4/50], Loss: 0.5701
Epoch [5/50], Loss: 0.4331
Epoch [6/50], Loss: 0.6260
Epoch [7/50], Loss: 0.5636
Epoch [8/50], Loss: 0.5083
Epoch [9/50], Loss: 0.5401
Epoch [10/50], Loss: 0.6103
Epoch [11/50], Loss: 0.5165
Epoch [12/50], Loss: 0.5954
Epoch [13/50], Loss: 0.4376
Epoch [14/50], Loss: 0.3187
Epoch [15/50], Loss: 0.3696
Epoch [16/50], Loss: 0.6463
Epoch [17/50], Loss: 0.5969
Epoch [18/50], Loss: 0.8233
Epoch [19/50], Loss: 0.6124
Epoch [20/50], Loss: 0.4607
Epoch [21/50], Loss: 0.3308
Epoch [22/50], Loss: 0.5175
Epoch [23/50], Loss: 0.2549
Epoch [24/50], Loss: 0.5101
Epoch [25/50], Loss: 0.3905
Epoch [26/50], Loss: 0.7831
Epoch [27/50], Loss: 0.1672
Epoch [28/50], Loss: 0.5866
Epoch [29/50], Loss: 0.5869
Epoch [30/50], Loss: 1.0875
Epoch [31/50], Loss: 0.3221
Epoch [32/50], Loss: 0.3308
Epoch [33/50], Loss: 0.5515
Epoch [34/50], Loss: 0.8740
Epoch [35/50], Loss: 0.3234
Epoch [36/50], Loss: 0.2663
E

In [18]:
torch.save(model.state_dict(), 'models/ titanic_model.pth')
print("Model saved to titanic_model.pth")

Model saved to titanic_model.pth


In [20]:
import torch.onnx
# Export the model to ONNX format
dummy_input = torch.randn(1, 7)  # Adjust the input size to match the number of features    
torch.onnx.export(model, dummy_input, "models/titanic_model.onnx", 
                  input_names=['input'], output_names=['output'], 
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
print("Model exported to titanic_model.onnx")


Model exported to titanic_model.onnx
